# Deep MNIST for Experts

What we will accomplish in this tutorial:

- Create a softmax regression function that is a model for recognizing MNIST digits, based on looking at every pixel in the image
- Use Tensorflow to train the model to recognize digits by having it "look" at thousands of examples (and run our first Tensorflow session to do so)
- Check the model's accuracy with our test data
- Build, train, and test a multilayer convolutional neural network to improve the results

本文的资料来自于 [tensorflow tutorial](https://www.tensorflow.org/get_started/mnist/pros)


## Setup

读入模块和数据

In [1]:
# 读入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 读入模块
import tensorflow as tf
sess = tf.InteractiveSession()


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


 If you are not using an InteractiveSession, then you should build the entire computation graph before starting a session and launching the graph.

# Part I: 简单的 Softmax regression 模型

## 1. Build a Softmax Regression Model

In this section we will build a softmax regression model with a single linear layer. In the next section, we will extend this to the case of softmax regression with a multilayer convolutional network.

In [2]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

1. 为什么是 784？ 
2. shape 的第一个维数为什么用 None?

In [3]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

y = tf.matmul(x,W) + b
print (y, y_)
print (x)

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


Tensor("add:0", shape=(?, 10), dtype=float32) Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
Tensor("Placeholder:0", shape=(?, 784), dtype=float32)


## 2. 模型训练和计算

In [8]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
#     print (batch.shape)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [12]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# print(sess.run(accuracy))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9213


# Part II: 多层卷积网路

### Weight Initialization

In [1]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling

In [21]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



## 1. Model building

In [27]:
# First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second Convolutional Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected Layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout Layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## 2. Train and Evaluate the Model

In [30]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
#         if i % 100 == 0:
#             train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
#             print('step %d, training accuracy %g' % (i, train_accuracy))
#             train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print('test accuracy %g' % accuracy.eval(feed_dict={
  x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.2
step 100, training accuracy 0.18
step 200, training accuracy 0.08
step 300, training accuracy 0.1
step 400, training accuracy 0.1
step 500, training accuracy 0.06
step 600, training accuracy 0.1
step 700, training accuracy 0.12
step 800, training accuracy 0.14
step 900, training accuracy 0.1
step 1000, training accuracy 0.08
step 1100, training accuracy 0.14
step 1200, training accuracy 0.16
step 1300, training accuracy 0.14
step 1400, training accuracy 0.16
step 1500, training accuracy 0.2
step 1600, training accuracy 0.38
step 1700, training accuracy 0.24
step 1800, training accuracy 0.24
step 1900, training accuracy 0.24
step 2000, training accuracy 0.28
step 2100, training accuracy 0.32
step 2200, training accuracy 0.38
step 2300, training accuracy 0.3
step 2400, training accuracy 0.38
step 2500, training accuracy 0.38
step 2600, training accuracy 0.34
step 2700, training accuracy 0.42
step 2800, training accuracy 0.34
step 2900, training accuracy 0.24